In [ ]:
from train import train
from config import Config

class CFG(Config):
    base_model = 'efficientnet_b0'   # resnet18/34/50, efficientnet_b0/b1/b2/b3/b4, efficientnet_v2_s, convnext_tiny, swin_t
    batch_size = 16
    epochs = 3
    base_lr = 1e-3
    scheduler_step_size = 2
    optimizer = 'AdamW'
    scheduler = 'StepLR'
    loss = 'KLDivLoss'
    lr_gamma = 0.1
    sgd_momentum = 0.9
    freeze_epochs = 0
    spec_random_trial_num = 1
    eeg_random_trial_num = 1
    data_type = 'eeg' # 'spec', 'eeg_tf', 'spec+eeg_tf'
    eeg_tf_data = 'eeg_tf_data_globalnorm'
    train_type = 'rater_num_split'
    init_epochs = 1
    wavenet_params = dict(eeg_ch=18, dropout=0.0, hidden_features=64)
    filt_hp = 0.5
    drop_ecg = False
    notes = ''

    # Augmentation
    use_mixup = False
    mixup_alpha = 1.0
    coarse_dropout_args = dict(p=0.5, max_holes=8, max_height=128, max_width=128)
    xymasking_args = dict(p=0.5, num_masks_y=(2, 4), mask_y_length=(500, 2000))
    # horizontal_flip_p = 0.5

    if data_type == 'spec':
        in_channels = 1
        spec_trial_selection = 'first'
        eeg_trial_selection = 'all'
    elif data_type == 'eeg_tf' or data_type == 'eeg':
        in_channels = 1
        spec_trial_selection = 'all'
        eeg_trial_selection = 'first'
    elif data_type == 'spec+eeg_tf' or data_type == 'spec+eeg_tf+eeg':
        spec_trial_selection = 'all'
        eeg_trial_selection = 'first'

for xymasking_args in [dict(p=0.25, num_masks_y=(2, 4), mask_y_length=(500, 2000)),
                       dict(p=0.75, num_masks_y=(2, 4), mask_y_length=(500, 2000)),
                       dict(p=0.1, num_masks_y=(2, 4), mask_y_length=(500, 2000)),
                       dict(p=0.4, num_masks_y=(2, 4), mask_y_length=(500, 2000)),
                       dict(p=0.6, num_masks_y=(2, 4), mask_y_length=(500, 2000)),
                       dict(p=0.9, num_masks_y=(2, 4), mask_y_length=(500, 2000))]:
    CFG.xymasking_args = xymasking_args
    for scheduler in ['StepLR']:
        CFG.scheduler = scheduler
        for epochs in [10]:
            CFG.epochs = epochs
            for eeg_tf_data in ['eeg_tf_data_globalnorm']:
                CFG.eeg_tf_data = eeg_tf_data
                for base_model in ['efficientnet_b0']:
                    CFG.base_model = base_model
                    for batch_size in [16]:
                        CFG.batch_size = batch_size
                        for init_epochs in [5]:
                            CFG.init_epochs = init_epochs
                            train(CFG)